In [ ]:
import torch
import torch.nn as nn

class TimeSeriesModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(TimeSeriesModel, self).__init__()

        # Capa de entrada lineal
        self.input_layer = nn.Linear(input_size, hidden_size)

        # Otras capas de tu modelo, por ejemplo, LSTM o más capas lineales

        # Capa de salida lineal
        self.output_layer = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        # Aplicar la capa de entrada
        x = self.input_layer(x)

        # Aplicar otras capas de tu modelo, si las tienes

        # Aplicar la capa de salida
        x = self.output_layer(x)

        return x

# Supongamos que cada paso temporal tiene 3 variables (ventas, vacaciones, fin de semana)
input_size = 3
hidden_size = 64  # Tamaño arbitrario, puedes ajustarlo según tu problema
output_size = 1  # Por ejemplo, si estás prediciendo ventas del próximo día

# Crear una instancia del modelo
model = TimeSeriesModel(input_size, hidden_size, output_size)

# Ejemplo de cómo se usaría la capa de entrada con un lote de datos
# Supongamos que batch_size es el tamaño del lote y seq_length es la longitud de la secuencia (30 en tu caso)
batch_size = 16
seq_length = 30

# Crear un tensor de ejemplo con dimensiones (batch_size, seq_length, input_size)
input_data = torch.randn((batch_size, seq_length, input_size))

# Pasar el tensor a través del modelo
output = model(input_data)

# Imprimir la forma del tensor de salida
print(output.shape)


In [ ]:
import torch
import torch.nn as nn

class RedNeuronal(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(RedNeuronal, self).__init()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        # x tiene forma (batch_size, pasos_temporales, num_variables)
        output, _ = self.lstm(x)
        # Tomar el último paso temporal para cada muestra en el lote
        output = output[:, -1, :]
        output = self.fc(output)
        return output

# Definir las dimensiones de entrada y salida
input_size = 3  # número de variables
hidden_size = 64  # tamaño oculto de la capa LSTM
num_layers = 2  # número de capas LSTM apiladas
output_size = 1  # salida de la red (por ejemplo, predicción de ventas)

# Crear la instancia de la red
modelo = RedNeuronal(input_size, hidden_size, num_layers, output_size)

# Crear datos de ejemplo (reemplace con sus propios datos)
datos_de_entrada = torch.randn(32, 30, 3)  # batch_size=32, pasos_temporales=30, num_variables=3

# Pasar los datos a través de la red
resultado = modelo(datos_de_entrada)


In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split
from lightgbm import LGBMRegressor
from datetime import datetime, timedelta
import numpy as np

# Generar datos aleatorios
np.random.seed(42)

productos = ['Producto_A', 'Producto_B', 'Producto_C']
fechas = [datetime(2023, 1, 1) + timedelta(days=i) for i in range(30)]
dias_de_semana = [fecha.weekday() < 5 for fecha in fechas]
ventas = np.random.randint(50, 100, size=(len(productos), len(fechas)))
vacaciones = np.random.choice([0, 1], size=(len(fechas),), p=[0.9, 0.1])

# Crear DataFrame
df = pd.DataFrame({
    'Producto': np.repeat(productos, len(fechas)),
    'Fecha': np.tile(fechas, len(productos)),
    'Ventas': ventas.flatten(),
    'Dia_Vacaciones': np.tile(vacaciones, len(productos)),
    'Fin_de_Semana': np.tile(dias_de_semana, len(productos))
})

# Ingeniería de características
df['Media_Ventas'] = df.groupby('Producto')['Ventas'].transform('mean')
df['Ventas_Anteriores'] = df.groupby('Producto')['Ventas'].shift(1)

# Dividir datos en entrenamiento y prueba
train, test = train_test_split(df, test_size=0.2, random_state=42)

train
# Separar características y objetivo
X_train = train[['Producto', 'Fecha', 'Media_Ventas', 'Dia_Vacaciones', 'Fin_de_Semana', 'Ventas_Anteriores']]
y_train = train['Ventas']

X_test = test[['Producto', 'Fecha', 'Media_Ventas', 'Dia_Vacaciones', 'Fin_de_Semana', 'Ventas_Anteriores']]
y_test = test['Ventas']

# Inicializar y entrenar el modelo LightGBM
modelo_lgbm = LGBMRegressor()
modelo_lgbm.fit(X_train, y_train)

# Realizar predicciones
predicciones = modelo_lgbm.predict(X_test)

# Evaluar el rendimiento del modelo (puedes utilizar métricas como MAE, MSE, etc.)
mae = np.mean(np.abs(predicciones - y_test))
print(f"Error absoluto medio (MAE): {mae}")


,Producto,Fecha,Ventas,Dia_Vacaciones,Fin_de_Semana,Media_Ventas,Ventas_Anteriores
0,Producto_A,2023-01-01,88,0,False,74.1,NaN
1,Producto_A,2023-01-02,78,0,True,74.1,88.0
2,Producto_A,2023-01-03,64,0,True,74.1,78.0
3,Producto_A,2023-01-04,92,0,True,74.1,64.0
4,Producto_A,2023-01-05,57,0,True,74.1,92.0
...,...,...,...,...,...,...,...
85,Producto_C,2023-01-26,65,0,True,74.4,70.0
86,Producto_C,2023-01-27,94,0,True,74.4,65.0
87,Producto_C,2023-01-28,67,0,False,74.4,94.0
88,Producto_C,2023-01-29,96,0,False,74.4,67.0
